#### Convolutional Neural Network

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [15]:
#!pip install Pillow
!pip install Pillow

In [1]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.2.0'

In [3]:
# Part 1 - Data Preprocessing
# Preprocessing the Train set

# Preprocessing the Training set
train_datagen = ImageDataGenerator(
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                  )

training_set = train_datagen.flow_from_directory('dog_vs_cat/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 1000 images belonging to 2 classes.


In [4]:

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dog_vs_cat/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 100 images belonging to 2 classes.


In [5]:
from tensorflow.keras.layers import Conv2D,MaxPool2D

In [7]:
Conv2D(padding="same")

TypeError: __init__() missing 2 required positional arguments: 'filters' and 'kernel_size'

In [6]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, padding="same", kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [7]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [8]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
32/32 [==============================] - 41s 1s/step - loss: 0.7258 - accuracy: 0.5110 - val_loss: 0.6911 - val_accuracy: 0.5200
Epoch 2/10
32/32 [==============================] - 25s 776ms/step - loss: 0.6888 - accuracy: 0.5520 - val_loss: 0.6781 - val_accuracy: 0.5800
Epoch 3/10
32/32 [==============================] - 7s 223ms/step - loss: 0.6790 - accuracy: 0.5640 - val_loss: 0.6857 - val_accuracy: 0.5100
Epoch 4/10
32/32 [==============================] - 7s 224ms/step - loss: 0.6673 - accuracy: 0.6070 - val_loss: 0.7200 - val_accuracy: 0.5000
Epoch 5/10
32/32 [==============================] - 7s 231ms/step - loss: 0.6776 - accuracy: 0.5710 - val_loss: 0.6359 - val_accuracy: 0.6600
Epoch 6/10
32/32 [==============================] - 7s 224ms/step - loss: 0.6616 - accuracy: 0.6000 - val_loss: 0.6763 - val_accuracy: 0.5600
Epoch 7/10
32/32 [==============================] - 7s 222ms/step - loss: 0.6313 - accuracy: 0.6470 - val_loss: 0.6188 - val_accuracy: 0.6300
Epoch 8

In [9]:
# save it as a h5 file


from tensorflow.keras.models import load_model

cnn.save('model_rcat_dog.h5')

In [10]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model_rcat_dog.h5')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [14]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dog_vs_cat/test/dog/dog.875.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result     = cnn.predict(test_image)

(1, 64, 64, 3)

In [17]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dog_vs_cat/test/cat/cat.617.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result     = cnn.predict(test_image)

In [18]:
result

array([[0.3530426]], dtype=float32)

In [19]:
if result[0]<=0.5:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is cat
